In [ ]:
# File Authorship Information
__author__ = """Matteo Lulli, Antonino Marcianò, Emanuele Zappalà"""
__copyright__ = """Copyright 2023, Matteo Lulli, Antonino Marcianò, Emanuele Zappalà, idea.deploy"""
__license__ = """Permission is hereby granted, free of charge, 
to any person obtaining a copy of this software and associated 
documentation files (the "Software"), to deal in the Software 
without restriction, including without limitation the rights to 
use, copy, modify, merge, publish, distribute, sublicense, 
and/or sell copies of the Software, 
and to permit persons to whom the Software is furnished to do so, 
subject to the following conditions:
The above copyright notice and this permission notice shall be 
included in all copies or substantial portions of the Software.
THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, 
EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES 
OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND 
NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT 
HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, 
WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER 
DEALINGS IN THE SOFTWARE."""
__maintainer__ = "Matteo Lulli, Emanuele Zappalà"
__email__ = "matteo.lulli@gmail.com"
__status__ = "Development"

In [ ]:
# Development cell
%load_ext autoreload
%autoreload 2

# Exact Evaluation of Hexagonal Spin-networks and Topological Quantum Neural Networks

Authors: Matteo Lulli (1), Antonino Marcianò (2, 3, 4), Emanuele Zappalà (5)

(1) Department of Mechanics and Aerospace Engineering, Southern University of Science and Technology, Shenzhen, Guangdong 518055, China, [lulli@sustech.edu.cn](mailto:lulli@sustech.edu.cn)

(2) Department of Physics, Fudan University, 200433 Shanghai, China, [marciano@fudan.edu.cn](mailto:marciano@fudan.edu.cn)

(3) Laboratori Nazionali di Frascati INFN, Frascati (Rome), Italy, EU

(4) INFN sezione Roma Tor Vergata, I-00133 Rome, Italy, EU

(5) Department of Mathematics and Statistics, Idaho State University Physical Science Complex |  921 S. 8th Ave., Stop 8085 | Pocatello, ID 83209, USA, [emanuelezappala@isu.edu](mailto:emanuelezappala@isu.edu)

**Abstract:**
The physical scalar product between spin-networks has been shown to be a fundamental tool in the theory of topological quantum neural networks (TQNN), which are quantum neural networks previously introduced by the authors in the context of quantum machine learning. However, the effective evaluation of the scalar product remains a bottleneck for the applicability of the theory.
We introduce an algorithm for the evaluation of the physical scalar product defined by Noui and Perez between spin-network with hexagonal shape. By means of recoupling theory and the properties of the Haar integration we obtain an efficient algorithm, and provide several proofs regarding the main steps. We investigate the behavior of the TQNN evaluations on certain classes of spin-networks with the classical and quantum recoupling. All results can be independently reproduced through the ``idea.deploy" framework [https://github.com/lullimat/idea.deploy](https://github.com/lullimat/idea.deploy)

# Reproducibility

This document is intended for those interested readers who want to reproduce the results reported in the paper [https://arxiv.org/abs/2310.03632](https://arxiv.org/abs/2310.03632). In the present case the computational resources needed should be available in general. 

Next development steps will include a class to measure the time required by each cell and output it in a .json file which can be sent to [matteo.lulli@gmail.com](mailto:matteo.lulli@gmail.com) and [emanuelezappala@isu.edu](mailto:emanuelezappala@isu.edu) so that average execution times will be available and organized according to the hardware.

Each subsection can be executed independently and reproduce the results which will be stored locally in the directory 'reproduced-data', so that the data will be generated only once.

Plots can also be generated using the same scripts employed for the figures of the paper. Since there are some issues in executing these scripts in the Jupyter environment we include them as separated files which are called from the cells themselves. **In order to reproduce the plots a working 'latex' installation is necessary to be present on the system.**

This file will be kept updated for new local features and developments in the parent project [**idea.deploy**](https://github.com/lullimat/idea.deploy)

This file is supposed to be pulled from the repository [https://arxiv.org/abs/2310.03632](https://arxiv.org/abs/2310.03632), from within the "papers" directory the idea.deploy project.

# Paper Results

In [ ]:
# Memory Management Class
import sys
sys.path.append("../../")

import sympy as sp
from idpy.SpinNetworks.Evaluations import Delta_n, Theta
from idpy.SpinNetworks.Honeycombs import GetNBYN, GetTetrahedron, PrintEdgesLabels, PlotColors, FullReduction
from idpy.SpinNetworks.PerezNouiProjector import PerezNouiAmplitude, PerezNouiProbability
from idpy.SpinNetworks.Evaluations import Q

from idpy.Utils.ManageData import ManageData

import h5py

class MDExhaustiveHexagons(ManageData):
    def __init__(self, N):
        ManageData.__init__(self, 
                            dump_file='transition_matrix_N' + str(N) + '.hdf5')

    def PushEvaluationNTransitionMatrix(self, evaluations, configurations,
                                        matrix, c_min, c_max, q_val):
        self.PushData(data=evaluations, 
                      key='c_min_' + str(c_min) + '_c_max_' + str(c_max) + '_q_' + str(q_val) + \
                          '/evaluations')
        self.PushData(data=configurations, 
                      key='c_min_' + str(c_min) + '_c_max_' + str(c_max) + '_q_' + str(q_val) + \
                          '/configurations')        
        self.PushData(data=matrix, 
                      key='c_min_' + str(c_min) + '_c_max_' + str(c_max) + '_q_' + str(q_val) + \
                      '/transition_matrix')
        self.Dump(kind='hdf5')

    def GetTransitionMatrix(self, c_min, c_max, q_val):
        return self.Read(kind='hdf5', 
                         full_key=\
                             self.__class__.__name__ + 
                             '/c_min_' + str(c_min) + 
                             '_c_max_' + str(c_max) +
                             '_q_' + str(q_val) + 
                             '/transition_matrix')

    def CheckForTransitionMatrix(self, c_min, c_max, q_val):
        key = self.__class__.__name__ + '/c_min_' + str(c_min) + '_c_max_' + str(c_max) + \
              '_q_' + str(q_val) + '/transition_matrix'
        return self.IsThereKey(key=key, kind='hdf5')

    def CheckForTransitionMatrixOld(self, c_min, c_max, q_val):
        h5_file = h5py.File(self.dump_file, 'r+')
        key = self.__class__.__name__ + '/c_min_' + str(c_min) + '_c_max_' + str(c_max) + \
              '_q_' + str(q_val) + '/transition_matrix'
        is_there_key = key in h5_file.keys()
        h5_file.close()
        return is_there_key

    def GetEvaluations(self, c_min, c_max, q_val):
        return self.Read(kind='hdf5', 
                         full_key=\
                             self.__class__.__name__ + 
                             '/c_min_' + str(c_min) + 
                             '_c_max_' + str(c_max) + 
                             '_q_' + str(q_val) + 
                             '/evaluations')

    def GetEvaluations(self, c_min, c_max, q_val):
        return self.Read(kind='hdf5', 
                         full_key=\
                             self.__class__.__name__ + 
                             '/c_min_' + str(c_min) + 
                             '_c_max_' + str(c_max) + 
                             '_q_' + str(q_val) + 
                             '/configurations')

    def IsFile(self):
        return os.path.isfile(self.dump_file)

## Finding all the cycles
- The cell below finds all the cycles up to $N=4$
- It also computes all possible configurations obtained by "summing" cycles up to a maximum number contained in the dict ```colors_ranges```
- These calculations prove that $N=2$ with $c_m=1$ and $c_M=6$, as well as $N=3$ with $c_m=1$ and $c_M=2$ are actually the optimal values - the transition matrix still fits into commonly available RAM memory requiring roughly 6GB for each value of $N$.

In [ ]:
colors_ranges = {2: {'min': 1, 'max': 6}, 
                 3: {'min': 1, 'max': 2}, 
                 4: {'min': 1, 'max': 2}}

N_min, N_max = 2, 4
for N in range(N_min, N_max + 1):
    print("Honeycomb lattice of size N:", N)
    honey = GetNBYN(N)
    print("Getting all graph cycles...")
    honey_cycles = honey.GetAllCycles(dump_file='honeycombs-cycles-paper.json')
    honey.InitAllCyclesLength()
    print("Elapsed time:", 
          honey.cycles_elapsed_time['d'], "d, ",
          honey.cycles_elapsed_time['h'], "h, ", 
          honey.cycles_elapsed_time['m'], "m, ", 
          honey.cycles_elapsed_time['s'], "s")
    print()
    
    tot = 0
    for M in honey.cycles_edges_ldict:
        tot += len(honey.cycles_edges_ldict[M])
    print("The total number of cycles is:", tot)
    tot = honey.GetCountCyclesCombinations(
        colors_ranges[N]['min'], colors_ranges[N]['max'], print_flag=True
    )
    print("The total number of states is:", tot)
    print("Transition matrix size:", 
          ((tot ** 2) * 4) / (2 ** 30), "GB")

    print()
    print()

## Transition Matrix Computations - $N=2,3$

- In the cell below we compute ateh transition matrix and write it disk
- Rouhly 12GB of disk need to be available

In [ ]:
## Computing and dumping transition matrix and plotting the data
import numpy as np
import os
from pathlib import Path
from idpy.Utils.SimpleTiming import SimpleTiming
from tqdm.notebook import tqdm

from IPython.display import display, Latex, Math
from idpy.Utils.Combinatorics import PrintMultinomial

q = Q()
_st = SimpleTiming()

colors_ranges = {2: {'min': 1, 'max': 6}, 
                 3: {'min': 1, 'max': 2}}

N_min, N_max = 2, 3
for N in range(N_min, N_max + 1):
    md, key_flag = MDExhaustiveHexagons(N=N), True
    if md.IsFile():
        print("File", md.dump_file, "found!")
        print("Looking for the transition matrix...")

        if not md.CheckForTransitionMatrix(
            c_min=colors_ranges[N]['min'], 
            c_max=colors_ranges[N]['max'], q_val=q()):
            key_flag = False
            print("colors combination not found!")
        else:
            print("found!")
    
    if not md.IsFile() or not key_flag:
        if not md.IsFile():
            print("File", md.dump_file, "not found!")
    
        print("Computing teh transition matrix for N:", N, 
              "c_min:", colors_ranges[N]['min'], "c_max:", colors_ranges[N]['max'])
        
        honey = GetNBYN(N)
        honey_cycles = honey.GetAllCycles(dump_file='honeycombs-cycles-paper.json')
        honey.InitAllCyclesLength()
        
        FullReduction(honey)
    
        ## We should associate a label to each combination
        ccombinations_count = \
            honey.GetCountCyclesCombinations(colors_ranges[N]['min'], colors_ranges[N]['max'])
        ccombinations, clabels = \
            honey.GetCyclesCombinations(colors_ranges[N]['min'], colors_ranges[N]['max'])
        
        evaluations, configurations = [], []
        
        print("Evaluating all configurations with colors in the range:", 
              [colors_ranges[N]['min'], colors_ranges[N]['max']])
        
        _st.Start()

        print("Evaluating the Spin-Network configurations")
        for cc_i, combinations in tqdm(enumerate(ccombinations), total=len(ccombinations)):
            for c_i, combination in tqdm(enumerate(combinations), total=len(combinations)):
                honey.ResetColors()
                honey.SetLoopColors([k * honey.cycles_edges[k_i] for k_i, k in enumerate(combination)])
                evaluation = complex(honey.GetEvaluation(q=q()).evalf())
                evaluations += [evaluation]
                configurations += (cc_i, c_i)
        _st.End()
        _st.PrintElapsedTime()
        print("\n")
        
        print("Computing transition matrix...")
        _st.Start()
        transition_matrix = np.zeros([ccombinations_count, ccombinations_count], dtype=np.float32)

        tm = []
        for v_i in tqdm(range(len(evaluations))):
            for v_j in range(v_i, len(evaluations)):
                value_i, value_j = evaluations[v_i], evaluations[v_j]
                probability = np.real(PerezNouiProbability(value_i, value_j))
                transition_matrix[v_i, v_j] = probability 
                transition_matrix[v_j, v_i] = probability
        
        _st.End()
        _st.PrintElapsedTime()

        print("Dumping data in file:", md.dump_file)

        evaluations = np.array(evaluations, dtype=np.csingle)
        configurations = np.array(configurations)        
        
        md.PushEvaluationNTransitionMatrix(evaluations=evaluations,
                                           configurations=configurations,
                                           matrix=transition_matrix, 
                                           c_min=colors_ranges[N]['min'], 
                                           c_max=colors_ranges[N]['max'],
                                           q_val=q())

## Figures
### Figure 15

In [ ]:
## Figure 15 is saved as './reproduced-figures/AR2.[png,pdf]'
import numpy as np
import os
from pathlib import Path
from idpy.Utils.SimpleTiming import SimpleTiming
from tqdm.notebook import tqdm

from IPython.display import display, Latex, Math
from idpy.Utils.Combinatorics import PrintMultinomial

colors_ranges = {2: {'min': 1, 'max': 6}, 
                 3: {'min': 1, 'max': 2}}

N = 2
q = Q()

md, key_flag = MDExhaustiveHexagons(N=N), True
if md.IsFile():
    print("File", md.dump_file, "found!")
    print("Looking for the transition matrix...")

    if not md.CheckForTransitionMatrix(
        c_min=colors_ranges[N]['min'], 
        c_max=colors_ranges[N]['max'], q_val=q()):
        key_flag = False
        print("colors combination not found!")
    else:
        print("found!")

if key_flag:
    '''
    At this point we can try to plot
    '''
    transition_matrix = \
        md.GetTransitionMatrix(
            c_min=colors_ranges[N]['min'], 
            c_max=colors_ranges[N]['max'], q_val=q()
        )

    sorted_cols = np.argsort(np.sum(transition_matrix, axis=0))
    sorted_transition_matrix = transition_matrix[:,sorted_cols]

    sorted_rows = np.argsort(np.sum(sorted_transition_matrix, axis=1))
    sorted_transition_matrix = sorted_transition_matrix[sorted_rows,:]

    from pathlib import Path
    
    reproduced_figures = Path('./reproduced-figures')
    if not reproduced_figures.is_dir():
        reproduced_figures.mkdir()

    %matplotlib inline    
    import matplotlib.pyplot as plt
    from matplotlib import rc, rcParams
    from idpy.Utils.Plots import SetAxPanelLabelCoords, SetMatplotlibLatexParamas, CreateFiguresPanels
    
    SetMatplotlibLatexParamas([rc], [rcParams])
        
    _nx_fig, _ny_fig = 1, 3
    
    fig = CreateFiguresPanels(_nx=_nx_fig,_ny=_ny_fig, _x_size = 6.5, _y_size = 3)
    
    max_row = 2048
    
    panel_a = plt.subplot2grid((_ny_fig, _nx_fig), (0, 0), colspan = 1, rowspan = 2)
    im = panel_a.imshow(sorted_transition_matrix, origin='lower')
    fig.colorbar(im)
    
    panel_a.set_xlabel('$i_R$')
    panel_a.set_ylabel('$j_R$')
    panel_a.set_title('$\\mathcal{A}_{i_R j_R}$')
    
    panel_b = plt.subplot2grid((_ny_fig, _nx_fig), (2, 0), colspan = 1, rowspan = 1, sharex=panel_a)
    panel_b.plot(np.sum(sorted_transition_matrix, axis=0))
    panel_b.set_xlabel('$i_R$')
    panel_b.set_ylabel('$\\mathcal{S}_{i_R}$')
    
    plt.savefig(reproduced_figures / ('AR2.png'), bbox_inches = 'tight', dpi = 300)
    plt.savefig(reproduced_figures / ('AR2.pdf'), bbox_inches = 'tight', dpi = 300)

In [ ]:
## Figure 16 is saved as './reproduced-figures/AR3.[png,pdf]'
import numpy as np
import os
from pathlib import Path
from idpy.Utils.SimpleTiming import SimpleTiming
from tqdm.notebook import tqdm

from IPython.display import display, Latex, Math
from idpy.Utils.Combinatorics import PrintMultinomial

colors_ranges = {2: {'min': 1, 'max': 6}, 
                 3: {'min': 1, 'max': 2}}

N = 3
q = Q()

md, key_flag = MDExhaustiveHexagons(N=N), True
if md.IsFile():
    print("File", md.dump_file, "found!")
    print("Looking for the transition matrix...")

    if not md.CheckForTransitionMatrix(
        c_min=colors_ranges[N]['min'], 
        c_max=colors_ranges[N]['max'], q_val=q()):
        key_flag = False
        print("colors combination not found!")
    else:
        print("found!")

if key_flag:
    '''
    At this point we can try to plot
    '''
    transition_matrix = \
        md.GetTransitionMatrix(
            c_min=colors_ranges[N]['min'], 
            c_max=colors_ranges[N]['max'], q_val=q()
        )

    sorted_cols = np.argsort(np.sum(transition_matrix, axis=0))
    sorted_transition_matrix = transition_matrix[:,sorted_cols]

    sorted_rows = np.argsort(np.sum(sorted_transition_matrix, axis=1))
    sorted_transition_matrix = sorted_transition_matrix[sorted_rows,:]

    from pathlib import Path
    
    reproduced_figures = Path('./reproduced-figures')
    if not reproduced_figures.is_dir():
        reproduced_figures.mkdir()

    %matplotlib inline    
    import matplotlib.pyplot as plt
    from matplotlib import rc, rcParams
    from idpy.Utils.Plots import SetAxPanelLabelCoords, SetMatplotlibLatexParamas, CreateFiguresPanels
    
    SetMatplotlibLatexParamas([rc], [rcParams])
        
    _nx_fig, _ny_fig = 1, 3
    
    fig = CreateFiguresPanels(_nx=_nx_fig,_ny=_ny_fig, _x_size = 6.5, _y_size = 3)
    
    max_row = 2048
    
    panel_a = plt.subplot2grid((_ny_fig, _nx_fig), (0, 0), colspan = 1, rowspan = 2)
    im = panel_a.imshow(sorted_transition_matrix, origin='lower')
    fig.colorbar(im)
    
    panel_a.set_xlabel('$i_R$')
    panel_a.set_ylabel('$j_R$')
    panel_a.set_title('$\\mathcal{A}_{i_R j_R}$')
    
    panel_b = plt.subplot2grid((_ny_fig, _nx_fig), (2, 0), colspan = 1, rowspan = 1, sharex=panel_a)
    panel_b.plot(np.sum(sorted_transition_matrix, axis=0))
    panel_b.set_xlabel('$i_R$')
    panel_b.set_ylabel('$\\mathcal{S}_{i_R}$')
    
    plt.savefig(reproduced_figures / ('AR3.png'), bbox_inches = 'tight', dpi = 300)
    plt.savefig(reproduced_figures / ('AR3.pdf'), bbox_inches = 'tight', dpi = 300)